In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras_tuner as kt
from tensorflow.keras.datasets import cifar10


In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from kerastuner import RandomSearch

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert class vectors to binary class matrices
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define predefined class labels for CIFAR-10
class_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


/var/folders/_7/q65t_5bs3653ct73dh3s10w00000gn/T/ipykernel_56636/3602459297.py:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [8]:
def build_model(hp):
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=(32, 32, 3)))
    
    # Hyperparameters
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=3, default=2)
    n_neurons = hp.Int("n_neurons", min_value=32, max_value=512, step=32)
    dropout_rate = hp.Float("dropout_rate", min_value=0.0, max_value=0.5, default=0.2)
    l2_reg = hp.Float("l2_reg", min_value=1e-4, max_value=1e-2, default=1e-4)
    momentum = hp.Float("momentum", min_value=0.5, max_value=0.99, default=0.9)
    
    # Add hidden layers based on hyperparameters
    for i in range(n_hidden):
        model.add(layers.Dense(units=n_neurons, activation='relu', kernel_regularizer=regularizers.l2(l2_reg)))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(dropout_rate))
    
    # Output layer
    model.add(layers.Dense(10, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="LOG"), momentum=momentum),
                 loss=tf.keras.losses.CategoricalCrossentropy(),
                 metrics=['accuracy'])

    return model


In [9]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld')

tuner.search_space_summary()

tuner.search(x_train, y_train,
             epochs=5,
             validation_data=(x_test, y_test))

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Best number of neurons in the first dense layer: {best_hps.get('n_neurons')}")
print(f"Best dropout rate: {best_hps.get('dropout_rate')}")
print(f"Best L2 regularization: {best_hps.get('l2_reg')}")
print(f"Best learning rate: {best_hps.get('learning_rate')}")
print(f"Best momentum: {best_hps.get('momentum')}")


TypeError: momentum is not a valid argument, kwargs should be empty  for `optimizer_experimental.Optimizer`.

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

# Define CIFAR-10 class labels
class_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Normalize pixel values to range 0 to 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Define model building function
def build_model(hp):
    model = keras.Sequential()

    # Tune the number of units in the first dense layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(layers.Flatten(input_shape=(32, 32, 3)))
    model.add(layers.Dense(units=hp_units, activation='relu'))

    # Tune the number of hidden layers and units in each hidden layer
    for i in range(hp.Int('num_layers', 1, 4)):
        model.add(layers.Dense(units=hp_units, activation='relu'))

    # Add output layer
    model.add(layers.Dense(10, activation='softmax'))

    # Tune the learning rate for the optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Instantiate the tuner and perform hyperparameter tuning
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Number of hyperparameter combinations to try
    executions_per_trial=3,  # Number of models to train per trial
    directory='my_dir',
    project_name='cifar10_ann_tuning')

# Perform the search for best hyperparameters
tuner.search(x_train, y_train,
             epochs=5,  # You can increase this for a more thorough search
             validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 55s]
val_accuracy: 0.2480333298444748

Best val_accuracy So Far: 0.47413333257039386
Total elapsed time: 00h 05m 13s


In [11]:
# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)

# Train the model with the best hyperparameters
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(x_test, y_test)

print("Test accuracy:", test_acc)

Epoch 1/10
1563/1563 [==============================] - 7s 4ms/step - loss: 1.8483 - accuracy: 0.3327 - val_loss: 1.7303 - val_accuracy: 0.3769
Epoch 2/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.6542 - accuracy: 0.4066 - val_loss: 1.6036 - val_accuracy: 0.4225
Epoch 3/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.5678 - accuracy: 0.4415 - val_loss: 1.5302 - val_accuracy: 0.4554
Epoch 4/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.5076 - accuracy: 0.4630 - val_loss: 1.4964 - val_accuracy: 0.4594
Epoch 5/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.4632 - accuracy: 0.4776 - val_loss: 1.4432 - val_accuracy: 0.4829
Epoch 6/10
1563/1563 [==============================] - 8s 5ms/step - loss: 1.4293 - accuracy: 0.4876 - val_loss: 1.4371 - val_accuracy: 0.4825
Epoch 7/10
1563/1563 [==============================] - 7s 4ms/step - loss: 1.3935 - accuracy: 0.5021 - val_loss: 1.5233 - val_accuracy:

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

# Define CIFAR-10 class labels
class_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Normalize pixel values to range 0 to 1
x_train, x_test = x_train / 255.0, x_test / 255.0


def build_model(hp):
  model = keras.Sequential()

  # Tune the number of units in the first dense layer
  hp_units = hp.Int('units', min_value=32, max_value=256, step=32)

  # Convolutional layers (alternative to Dense layer for feature extraction)
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(layers.BatchNormalization())  # Improve training stability
  model.add(layers.MaxPooling2D((2, 2)))

  # Tune the number of hidden layers
  for i in range(hp.Int('num_layers', 1, 3)):
    model.add(layers.Conv2D(hp_units, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

  # Flatten for dense layers
  model.add(layers.Flatten())

  # Dense layers with L2 regularization to prevent overfitting
  model.add(layers.Dense(hp_units, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)))
  model.add(layers.Dropout(0.2))  # Randomly drop neurons during training

  # Add output layer
  model.add(layers.Dense(10, activation='softmax'))

  # Tune the learning rate for the optimizer
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4])

  # Compile the model with Adam optimizer and early stopping
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  # Early stopping to prevent overfitting (monitor validation loss)
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

  return model

# Instantiate the tuner and perform hyperparameter tuning
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,  # Increased number of trials for more exploration
    executions_per_trial=3,  # Train 3 models per hyperparameter combination
    directory='my_dir',
    project_name='cifar10_cnn_tuning')

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# Perform the search for best hyperparameters
tuner.search(x_train, y_train,
             epochs=10,  # You can further increase epochs for a more thorough search
             validation_data=(x_test, y_test),
             callbacks=[early_stopping])  # Add early stopping during search

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)

# Train the model with the best hyperparameters
model.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test))  # Increased epochs for training

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(x_test, y_test)

print("Test accuracy:", test_acc)


Trial 12 Complete [00h 00m 00s]

Best val_accuracy So Far: 0.7378999988238016
Total elapsed time: 01h 50m 40s

Search: Running Trial #13

Value             |Best Value So Far |Hyperparameter
256               |160               |units
1                 |2                 |num_layers
0.0001            |0.001             |learning_rate



Epoch 1/10
1563/1563 [==============================] - 79s 50ms/step - loss: 1.9149 - accuracy: 0.5020 - val_loss: 1.5914 - val_accuracy: 0.6044
Epoch 2/10
1563/1563 [==============================] - 79s 51ms/step - loss: 1.4602 - accuracy: 0.6436 - val_loss: 1.4828 - val_accuracy: 0.6285
Epoch 3/10
1563/1563 [==============================] - 78s 50ms/step - loss: 1.1917 - accuracy: 0.7242 - val_loss: 1.3011 - val_accuracy: 0.6816
Epoch 4/10
1563/1563 [==============================] - 72s 46ms/step - loss: 0.9728 - accuracy: 0.7907 - val_loss: 1.2500 - val_accuracy: 0.7056
Epoch 5/10
1563/1563 [==============================] - 104s 66ms/step - loss: 0.7902 - accuracy: 0.8487 - val_loss: 1.2384 - val_accuracy: 0.7087
Epoch 6/10
1563/1563 [==============================] - 98s 63ms/step - loss: 0.6454 - accuracy: 0.8982 - val_loss: 1.2683 - val_accuracy: 0.7074
Epoch 7/10
1563/1563 [==============================] - 95s 61ms/step - loss: 0.5376 - accuracy: 0.9332 - val_loss: 1.2422 

Epoch 1/10
1563/1563 [==============================] - 63s 40ms/step - loss: 1.8891 - accuracy: 0.5123 - val_loss: 1.5475 - val_accuracy: 0.6183
Epoch 2/10
1563/1563 [==============================] - 83s 53ms/step - loss: 1.4369 - accuracy: 0.6519 - val_loss: 1.3522 - val_accuracy: 0.6736
Epoch 3/10
1563/1563 [==============================] - 70s 45ms/step - loss: 1.1851 - accuracy: 0.7232 - val_loss: 1.2732 - val_accuracy: 0.6887
Epoch 4/10
1563/1563 [==============================] - 81s 52ms/step - loss: 0.9856 - accuracy: 0.7826 - val_loss: 1.1866 - val_accuracy: 0.7117
Epoch 5/10
1563/1563 [==============================] - 80s 51ms/step - loss: 0.8066 - accuracy: 0.8411 - val_loss: 1.1823 - val_accuracy: 0.7180
Epoch 6/10
1563/1563 [==============================] - 94s 60ms/step - loss: 0.6664 - accuracy: 0.8856 - val_loss: 1.1729 - val_accuracy: 0.7252
Epoch 7/10
1563/1563 [==============================] - 95s 61ms/step - loss: 0.5585 - accuracy: 0.9224 - val_loss: 1.2202 -

KeyboardInterrupt: 